## Multirun experiment pipeline (WheatFspm)

The following notebook establishes a generalized pipeline for evaluating a computing reservoir against a given task, given multiple experimental runs of the same reservoir.


In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src

In [2]:
DATASET_NEMA_H0 = '../datasets/dataset_NEMA_NEMA_H0.csv'
DATASET_NEMA_H3 = '../datasets/dataset_NEMA_NEMA_H3.csv'
DATASET_NEMA_H15 = '../datasets/dataset_NEMA_NEMA_H15.csv'

### Loading the datasets

These datasets were collected and converted in the WheatFspm repository.

There are three simulations made available in the WheatFspm repository that are useable for RC experiments: NEMA H0, H3 and H15.

We can try using these datasets in two different ways:

1. Treat every dataset as a separate plant, training a readout for each simulation run.
2. Concatenating the three datasets as observed behavior of a single plant.

In [3]:
from src.model.rc_dataset import ExperimentDataset

dataset_nema_h0 = ExperimentDataset(csv_path=DATASET_NEMA_H0)
dataset_nema_h3 = ExperimentDataset(csv_path=DATASET_NEMA_H3)
dataset_nema_h15 = ExperimentDataset(csv_path=DATASET_NEMA_H15)

datasets = [
  ('NEMA_H0', dataset_nema_h0), 
  ('NEMA_H3', dataset_nema_h3), 
  ('NEMA_H15', dataset_nema_h15)
]

### Defining targets and observed state variables

These were selected in a previous notebook, `2022_03_23_wheatfspm_dataset_inspection.ipynb` and are defined in a config file for reuse among notebooks.

In [4]:
%reload_ext autoreload
%autoreload 2 

from model_config import targets, state_variables

print(f'Targets:')
for target in targets:
  print(f'\t- {target}')

print(f'\nState variables:')
for state_var in state_variables:
  print(f'\t- {state_var}')

Targets:
	- input_air_temperature
	- input_humidity
	- input_PARi
	- output__axes__Total_Transpiration
	- output__axes__C_exudated
	- output__axes__SAM_temperature
	- output__axes__delta_teq
	- output__axes__sum_respi_shoot
	- output__organ_roots__N_exudation

State variables:
	- state__An
	- state__Transpiration
	- state__S_Sucrose
	- state__Ts
	- state__gs
	- state__Ag
	- state__Tr
	- state__sucrose
	- state__Rd
	- state__sum_respi
	- state__Photosynthesis
	- state__PARa


### Data preprocessing, grouping and train-test splitting

The available datasets will be processed into 4 datasets:

- NEMA_H0
- NEMA_H3
- NEMA_H15
- NEMA_COMBINED (concatenated as data from the same plant)

In [5]:
from src.learning.preprocessing import generate_mask


WARMUP_STEPS = 4 * 24
DAY_MASK = generate_mask(5, 21)

In [6]:
from wheatfspm_pipeline_utils import preprocess_data, group_by_day, train_test_split_alternating
from wheatfspm_pipeline_utils import direct_target_generator, direct_reservoir_generator
from wheatfspm_pipeline_utils import preprocess_raw_X
from model_config import max_time_step

def generate_X_y_groups(datasets, target, state_var, target_generator, state_generator):
  """Generates X, y and groups arrays for each dataset, plus a concatenated dataset.
     NOTE: The groups in the concatenated dataset are such that the same calendar day is in the same group.

     Also generates a baseline dataset where the reservoir is just a combination of all environmental inputs.
  """
  data = {}

  # Preprocess the data for each dataset
  for name, dataset in datasets:
    target_data = next(target_generator(dataset, target, name))
    reservoir_data = next(state_generator(dataset, state_var, name))
    X_raw, y_raw = preprocess_data(dataset, target_data, reservoir_data, WARMUP_STEPS, DAY_MASK)
    X, y = X_raw[0, :, :], y_raw[0, :]
    groups = group_by_day(X, DAY_MASK)
    data[name] = (X, y, groups)

  # Generate the concatenated dataset
  all_arrays = list(data.values())
  X_combined = np.concatenate(list(map(lambda x : x[0], all_arrays)))
  y_combined = np.concatenate(list(map(lambda x : x[1], all_arrays)))
  groups_combined = np.concatenate(list(map(lambda x : x[2], all_arrays)))
  data['combined'] = (X_combined, y_combined, groups_combined)

  # Generate the baseline dataset
  # baseline_name, baseline_dataset = datasets[0]
  # input_vars = baseline_dataset.get_input_variables()
  # X_baseline = np.empty((max_time_step[baseline_name], len(input_vars)))
  # for i, var in enumerate(input_vars):
  #   X_baseline[:, i] = baseline_dataset.get_target(var, baseline_name)[:max_time_step[baseline_name]]
  # X_baseline = preprocess_raw_X(X_baseline, WARMUP_STEPS, DAY_MASK)
  # y_baseline = data[baseline_name][1]
  # groups_baseline = data[baseline_name][2]
  # data[f'baseline_{baseline_name}'] = X_baseline, y_baseline, groups_baseline

  return data

In [7]:
TARGET = 'output__axes__C_exudated' # targets[0]
STATE_VAR = 'state__Ts' # state_variables[0]

preprocessed_data = generate_X_y_groups(datasets, TARGET, STATE_VAR, direct_target_generator, direct_reservoir_generator)

for name, (X, y, groups) in preprocessed_data.items():
  print(f'{name}:')
  print(f'\tX: {X.shape}')
  print(f'\ty: {y.shape}')
  print(f'\tgroups: {len(np.unique(groups))} (shape {groups.shape})')

NEMA_H0:
	X: (400, 10)
	y: (400,)
	groups: 25 (shape (400,))
NEMA_H3:
	X: (496, 10)
	y: (496,)
	groups: 31 (shape (496,))
NEMA_H15:
	X: (528, 10)
	y: (528,)
	groups: 33 (shape (528,))
combined:
	X: (1424, 10)
	y: (1424,)
	groups: 33 (shape (1424,))


### Model definition

- Readout model is a standard RidgeRegression model with intercept term and CV-tuned regularization strength $\alpha$.
- CV search grid is a progression of logarithmicly spaced values for regularization strength $\alpha$.
- CV and testing metric is NMSE.

In [8]:
N_FOLDS = 3;

In [9]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold

from src.learning.scorers import nmse_scorer

# Define model
readout = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])

# define search grid
search_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]

# define cross-validation and testing metric
scorer = nmse_scorer

# Define CV fold strategy
folds = GroupKFold(n_splits=N_FOLDS)

### Generating a manifest of all experiments to run

Currently we are only benchmarking direct target prediction, but in the future there will be other tasks generated from the base targets as well. These will be generated in this section.


In [10]:
TARGETS = [(target_name, direct_target_generator) for target_name in targets]
STATE_VARS = [(state_var, direct_reservoir_generator) for state_var in state_variables]

TRAIN_TEST_RATIO = 1

### Fitting all readout functions

Process:

- For each target:
  - For each observed state variable:
    - For each dataset:
      1. Preprocess the data
      2. Fit for each dataset
      3. Store the resulting training, cross-validation and test scores.

In [11]:
from tqdm import tqdm
from src.learning.training import perform_gridsearch


total_loops = len(targets) * len(state_variables) * (len(preprocessed_data))
print(f'Performing {total_loops} fits...')


models = {}
results = []


with tqdm(total=total_loops) as pbar:

    for target_name, target_generator in TARGETS:
        for state_var, state_generator in STATE_VARS:

            # Preprocess data for model fit
            preprocessed_data = generate_X_y_groups(datasets, target_name, state_var, target_generator, state_generator)

            # For each dataset combination
            for dataset_name, (X, y, groups) in preprocessed_data.items():
                train, test = train_test_split_alternating(X, y, groups, ratio=TRAIN_TEST_RATIO)

                # fit model
                X_train, y_train, groups_train = train
                model, scores = perform_gridsearch(readout, X_train, y_train, groups_train, folds, search_grid, verbose=False)
                (train_mean, train_std), (cv_mean, cv_std) = scores

                # Determine test score
                X_test, y_test, _ = test
                test_score = scorer(model, X_test, y_test)

                # store results
                models[(target_name, state_var, dataset_name)] = model
                results.append({
                    'target': target_name,
                    'state_var': state_var,
                    'dataset': dataset_name,
                    'test_score': test_score,
                    'train_mean': train_mean,
                    'train_std': train_std,
                    'cv_mean': cv_mean,
                    'cv_std': cv_std
                })

                pbar.update(1)                

Performing 432 fits...


100%|██████████| 432/432 [01:30<00:00,  4.79it/s]


### Storing experiment results

Storing all the fit results into a Pandas dataframe, then storing it as a CSV file for further inspection in another notebook.

In [12]:
results_df = pd.DataFrame.from_dict(results)
results_df.set_index(['target', 'state_var', 'dataset'])
results_df.to_csv(f'test_results_sub24.csv')
results_df.sample(10)

,target,state_var,dataset,test_score,train_mean,train_std,cv_mean,cv_std
421,output__organ_roots__N_exudation,state__sum_respi,NEMA_H3,-0.199992,-0.117836,0.012084,-0.226898,0.073079
208,output__axes__C_exudated,state__gs,NEMA_H0,-0.442742,-0.420083,0.029570,-0.473613,0.054519
89,input_humidity,state__Photosynthesis,NEMA_H3,-0.608611,-0.604624,0.100007,-1.031751,0.319949
381,output__axes__sum_respi_shoot,state__PARa,NEMA_H3,-1.096394,-0.927982,0.045092,-1.094052,0.095030
212,output__axes__C_exudated,state__Ag,NEMA_H0,-0.426593,-0.523349,0.025387,-0.557789,0.040780
348,output__axes__sum_respi_shoot,state__Ts,NEMA_H0,-1.398232,-0.297419,0.027591,-0.424407,0.049004
115,input_PARi,state__gs,combined,-0.170378,-0.163310,0.013024,-0.224446,0.059889
266,output__axes__SAM_temperature,state__Tr,NEMA_H15,-0.248585,-0.131018,0.011436,-0.267087,0.062409
341,output__axes__sum_respi_shoot,state__Transpiration,NEMA_H3,-1.066146,-0.846099,0.061092,-1.013421,0.108423
309,output__axes__delta_teq,state__Ag,NEMA_H3,-0.672693,-0.514685,0.017826,-0.656568,0.049494
